In [1]:
# Задание: разработать рекомендательную систему, используя пакет surprise.

# Взять датает movielens 1M/

# Цель получите RMSE на тестовом сете 0,87 и ниже.

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [3]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-07-02 15:51:38--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.137.139, 74.125.137.113, 74.125.137.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-07-02 15:51:39--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.1s    

2024-07-02 15:51:41 (7.59 MB/s)

In [4]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [5]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [6]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [8]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)# делаем общую таблицу
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [9]:
movies_with_ratings.title.value_counts()# сколько людей поставили рейтинг

title
Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: count, Length: 9719, dtype: int64

In [ ]:
movies_with_ratings.userId.value_counts()#  проверка рейтингованности юзера.
#т.е. сколько раз он установл рейтинг

userId
414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: count, Length: 610, dtype: int64

In [11]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357246 sha256=11d2ac8137ae4f8d2b45cd8eef2b066daf3892bf68698c435328035de5c0049e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [12]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection.validation import cross_validate

In [13]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [19]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [14]:
len(dataset.index)

100836

In [15]:
rat_min = ratings.rating.min()
rat_max = ratings.rating.max()

In [16]:
reader = Reader(rating_scale=(rat_min, rat_max))
data = Dataset.load_from_df(dataset, reader)


In [17]:
algo = KNNWithMeans(k=100, sim_options={
    'name': 'msd',
    'user_based': True  # compute  similarities between users
})
#algo.fit(trainset)

In [18]:
# Кросс валидация в данном случае используется для определения метрики качества.
#т.е. мы задаем модель, а кросс валидация выдает метрику качества.  Она не учавствует в обучении модели.

model = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

In [20]:
model # по каждому фолду видно, что точность больше 0.89. Раз точность нас устраивает, можно обучить модель.

{'test_rmse': array([0.89483208, 0.89536403, 0.89258528, 0.90108837, 0.90070896]),
 'test_mae': array([0.68205698, 0.68332824, 0.68413338, 0.68685612, 0.68891276]),
 'fit_time': (0.20483827590942383,
  0.32972025871276855,
  0.4004361629486084,
  0.28436946868896484,
  0.1389024257659912),
 'test_time': (5.479786396026611,
  5.587741136550903,
  5.252286911010742,
  5.192198038101196,
  1.8058865070343018)}

In [21]:
model['test_rmse'].mean()

0.8969157424290135

In [ ]:
# Lets try to find best user_ based algirytm
similarities = ['cosine', 'msd', 'pearson', 'pearson_baseline']
best_model = []
best_accuracy = 5
best_similarity =''

for similarity in similarities:
  algo = KNNWithMeans(k=100, sim_options={
      'name': similarity,
      'user_based': True  # compute  similarities between users
  })

  model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

  accuracy = model['test_rmse'].mean()

  if accuracy < best_accuracy:
    best_accuracy = accuracy
    best_similarity = similarity

best_similarity, best_accuracy

('pearson_baseline', 0.8963787072440997)

In [ ]:
# lets find best count of neibows in user_based algorytm
best_neighbour_count =0
best_accuracy = 5



for i in range(10, 110, 10):
    algo = KNNWithMeans(i, sim_options={
      'name': 'pearson_baseline',
      'user_based': True  # compute  similarities between users
  })

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_neighbour = i

best_neighbour, best_accuracy

(50, 0.8950744803503289)

In [ ]:
# Lets try t find best item-based algirytm
similarities = ['cosine', 'msd', 'pearson', 'pearson_baseline']
best_model = []
best_accuracy = 5
best_similarity =''

for similarity in similarities:
  algo = KNNWithMeans(k=100, sim_options={
      'name': similarity,
      'user_based': False  # compute  similarities between users
  })

  model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

  accuracy = model['test_rmse'].mean()

  if accuracy < best_accuracy:
    best_accuracy = accuracy
    best_similarity = similarity

best_similarity, best_accuracy

('pearson_baseline', 0.8831904243022738)

In [ ]:
# lets find best count of neibows in item_based algorytm
best_neighbour_count =0
best_accuracy = 5



for i in range(10, 110, 10):
    algo = KNNWithMeans(i, sim_options={
      'name': 'pearson_baseline',
      'user_based': False  # compute  similarities between users
  })

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_neighbour = i

best_neighbour, best_accuracy

(50, 0.8816350445647485)

In [ ]:
# lets chage algorytm

# Lets try to find best item-based algirytm. Lets change algo to KNNBasic
similarities = ['cosine', 'msd', 'pearson', 'pearson_baseline']
best_model = []
best_accuracy = 5
best_similarity =''

for similarity in similarities:
  algo = KNNBasic(k=50, sim_options={
      'name': similarity,
      'user_based': False  # compute  similarities between users
  })

  model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

  accuracy = model['test_rmse'].mean()

  if accuracy < best_accuracy:
    best_accuracy = accuracy
    best_similarity = similarity

best_similarity, best_accuracy


('msd', 0.9073195523131062)

In [ ]:
from surprise import KNNWithZScore

# Lets try t find best item-based algirytm. Lets change algo to KNNwithZscore
similarities = ['cosine', 'msd', 'pearson', 'pearson_baseline']
best_model = []
best_accuracy = 5
best_similarity =''

for similarity in similarities:
  algo = KNNWithZScore(k=50, sim_options={
      'name': similarity,
      'user_based': False  # compute  similarities between users
  })

  model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

  accuracy = model['test_rmse'].mean()

  if accuracy < best_accuracy:
    best_accuracy = accuracy
    best_similarity = similarity

best_similarity, best_accuracy


('pearson_baseline', 0.8870225419086124)

In [ ]:
from surprise import CoClustering

# Lets try t find best item-based algirytm. Lets change algo to Coclustering
algo = CoClustering(n_cltr_u = 30, n_cltr_i = 30, n_epochs = 20, random_state=42, verbose =False)
model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)


model['test_rmse'].mean()


1.0052927575676733

In [ ]:
# lets find best  n_cltr_u  in coCLustering algorytm

from surprise import CoClustering

best_n_cltr_u =0
best_accuracy = 5



for i in range(10, 160, 10):

    algo = CoClustering(n_cltr_u = i, n_cltr_i = 30, n_epochs = 20, random_state=42, verbose =False)

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_n_cltr_u = i

best_n_cltr_u, best_accuracy

(10, 0.9768735067187784)

In [ ]:
# lets find best  n_cltr_i  in coCLustering algorytm

from surprise import CoClustering

best_n_cltr_u =0
best_accuracy = 5



for i in range(10, 120, 10):

    algo = CoClustering(n_cltr_u = 10, n_cltr_i = i, n_epochs = 20, random_state=42, verbose =False)

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_n_cltr_i = i

best_n_cltr_i, best_accuracy

(10, 0.9690680103193048)

In [ ]:
# lets find best epochs count in coCLustering algorytm

from surprise import CoClustering

best_n_cltr_u =0
best_accuracy = 5

for i in range(1, 10, 1):

    algo = CoClustering(n_cltr_u = 10, n_cltr_i = 10, n_epochs = i, random_state=42, verbose =False)

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_epochs = i

best_epochs, best_accuracy

(1, 0.9528143070042709)

In [23]:
# lets try algo SVDPP

from surprise import SVDpp

best_n_cltr_u =0
best_accuracy = 5

for i in range(1, 10, 1):

    algo = SVDpp(n_factors=20, n_epochs=i, init_mean=0, random_state=42, verbose =False)

    model = cross_validate(algo,data, measures = ['RMSE', 'MAE'], cv =5, verbose = False, n_jobs= -1)

    accuracy = model['test_rmse'].mean()

    if accuracy < best_accuracy:
      best_accuracy = accuracy
      best_epochs = i

best_epochs, best_accuracy

(9, 0.8699553405387933)

In [24]:
 # best model is
algo = SVDpp(n_factors=20, n_epochs=9, init_mean=0, random_state=42, verbose =False)
# RMSE<0.87

In [25]:
trainset = data.build_full_trainset() #На вход должен подаваться тренировочный сет.
algo.fit(trainset)

In [32]:
for i in range(10):
  print(algo.predict(uid=i, iid='Fight Club (1999)'))# Проверка подходит ли фильм юзеру


user: 0          item: Fight Club (1999) r_ui = None   est = 4.21   {'was_impossible': False}
user: 1          item: Fight Club (1999) r_ui = None   est = 5.00   {'was_impossible': False}
user: 2          item: Fight Club (1999) r_ui = None   est = 4.38   {'was_impossible': False}
user: 3          item: Fight Club (1999) r_ui = None   est = 3.20   {'was_impossible': False}
user: 4          item: Fight Club (1999) r_ui = None   est = 3.91   {'was_impossible': False}
user: 5          item: Fight Club (1999) r_ui = None   est = 4.22   {'was_impossible': False}
user: 6          item: Fight Club (1999) r_ui = None   est = 4.09   {'was_impossible': False}
user: 7          item: Fight Club (1999) r_ui = None   est = 3.88   {'was_impossible': False}
user: 8          item: Fight Club (1999) r_ui = None   est = 4.12   {'was_impossible': False}
user: 9          item: Fight Club (1999) r_ui = None   est = 4.07   {'was_impossible': False}
